In [137]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
ROOT = '/home/robert/DataSpellProjects/MLS'
CURRENT_YEAR = 2022

In [138]:
def split_score(score):
    split = score.split('-')
    if '(' not in score:
        return split[0], split[1]
    else:
        return split[0].split(') ')[1], split[1].split(' (')[0]

In [139]:
def get_winner(home_team, away_team, score):
    home_score, away_score = split_score(score)
    if home_score > away_score:
        return home_team
    elif away_score > home_score:
        return away_team
    # Return regular season Tie
    elif '(' not in score and home_score == away_score:
        return np.nan
    elif '(' in score and home_score == away_score:
        home_pen = score.split('-')[0]
        home_pen = int(re.search('\(([^)]+)', home_pen).group(1))
        away_pen = score.split('-')[1]
        away_pen = int(re.search('\(([^)]+)', away_pen).group(1))
        if home_pen > away_pen:
            return home_team
        else:
            return away_team

In [140]:
def generate_games_played(code, year):
    """Giver a year and code, this script will go scrape sports-reference, create a csv, and return a df"""
    if year==CURRENT_YEAR:
        url = 'https://fbref.com/en/comps/22/schedule/Major-League-Soccer-Scores-and-Fixtures'
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        table = soup.find_all('table', {'id': 'sched_11499_1'})
        df = pd.read_html(str(table))[0]
        if 'Round' not in list(df.columns):
            df['Round']='Regular Season'
        df = df[df.Score != 'Score']
    else:
        url = f'https://fbref.com/en/comps/22/{code}/schedule/{year}-Major-League-Soccer-Scores-and-Fixtures'
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        table = soup.find_all('table', {'id': 'sched_all'})
        df = pd.read_html(str(table))[0]

    try:
        columns = ['Round', 'Date', 'Home', 'xG', 'Score', 'xG.1', 'Away', 'Attendance']
        df = df[columns]
    except KeyError:
        columns = ['Round', 'Date', 'Home', 'Score', 'Away', 'Attendance']
        df = df[columns]
    df['GHome'] = np.nan
    df['GAway'] = np.nan
    df['Winner'] = np.nan
    df['Season'] = np.nan
    for i in df.index:
        try:
            df.at[i, 'Score']=df.at[i, 'Score'].replace('–', '-')
        except AttributeError:
            df.drop([i], inplace=True)
            continue
        if df.at[i, 'Round']=='Round':
            df.drop([i], inplace=True)
            continue
        df.at[i, 'Season'] = year
        df.at[i, 'GHome'], df.at[i, 'GAway'] = split_score(df.at[i, 'Score'])
        df.at[i, 'Winner'] = get_winner(df.at[i, 'Home'], df.at[i, 'Away'], df.at[i, 'Score'])
    df['GHome'] = df.GHome.astype('int64')
    df['GAway'] = df.GAway.astype('int64')
    df['Season'] = df.Season.astype('int64')
    df['Attendance'].fillna(0, inplace=True)
    df['Attendance'] = df.Attendance.astype('int64')
    df.rename(columns={'xG': 'xGH', 'xG.1':'xGA'}, inplace=True)
    df.drop(['Score'], axis=1, inplace=True)
    df.to_csv(f'{ROOT}/Results/{year}.csv')
    return df

In [141]:
url_codes = {2022:0, 2021: 11006, 2020: 10090, 2019:2798, 2018:1759, 2017:1558, 2016:1503, 2015:1369, 2014:708, 2013:643, 2012:577, 2011:509, 2010:442, 2009:374, 2008:316, 2007:260, 2006:211, 2005:168, 2004:133, 2003:100, 2002:75, 2001:56, 2000:44, 1999:37, 1998:34, 1997: 32, 1996: 30}

In [142]:
AllYears = pd.DataFrame()
for year in url_codes:
    df = generate_games_played(url_codes[year], year)
    df = df[['Season','Round', 'Date', 'Home', 'GHome', 'GAway', 'Away','Winner']]
    AllYears = pd.concat([AllYears, df], ignore_index=True)
AllYears.to_csv(f'{ROOT}/Results/AllYears.csv')

In [143]:
AllYears

,Season,Round,Date,Home,GHome,GAway,Away,Winner
0,2022,Regular Season,2022-02-26,Philadelphia,1,1,Minnesota Utd,NaN
1,2022,Regular Season,2022-02-26,Los Angeles FC,3,0,Colorado Rapids,Los Angeles FC
2,2022,Regular Season,2022-02-26,Columbus Crew,4,0,Vancouver,Columbus Crew
3,2022,Regular Season,2022-02-26,FC Dallas,1,1,Toronto FC,NaN
4,2022,Regular Season,2022-02-26,San Jose,1,3,NY Red Bulls,NY Red Bulls
...,...,...,...,...,...,...,...,...
7020,1996,Conference Finals,1996-10-10,LA Galaxy,2,1,KC Wiz,LA Galaxy
7021,1996,Conference Finals,1996-10-10,D.C. United,4,1,Tampa Bay,D.C. United
7022,1996,Conference Finals,1996-10-12,Tampa Bay,1,2,D.C. United,D.C. United
7023,1996,Conference Finals,1996-10-13,KC Wiz,1,1,LA Galaxy,LA Galaxy
